### Import of necessary libraries

In [1]:
from interface import Interface
from shapely import Polygon
import folium
import geopandas
import json

### Creation of a polygon as an example of AOI

In [2]:
# simplified example of a call to the interface

lat_point_list = [50.854457, 50.924457, 52.518172, 50.072651, 48.853033]
lon_point_list = [4.377184, 4.377184, 13.407759, 14.435935, 2.349553]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))

polygon_geom = geopandas.GeoSeries(polygon_geom)

m = folium.Map(location=[50.854457, 4.377184], zoom_start=5)
folium.GeoJson(polygon_geom.__geo_interface__).add_to(m)

x = geopandas.GeoSeries(polygon_geom)
              
rectangle = geopandas.GeoDataFrame(geometry=x.envelope)

# Define the number of rows and columns for the grid
num_rows = 5
num_cols = 5

# Calculate the width and height of each sub-area
width = (rectangle.bounds['maxx'] - rectangle.bounds['minx']) / num_cols
height = (rectangle.bounds['maxy'] - rectangle.bounds['miny']) / num_rows

# Create an empty list to store the sub-areas
sub_areas = []

# Iterate over the rows and columns to create the grid of sub-areas
for row in range(num_rows):
    for col in range(num_cols):
        minx = rectangle.bounds['minx'] + col * width
        maxx = minx + width
        miny = rectangle.bounds['miny'] + row * height
        maxy = miny + height
        sub_area_coords = [(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy)]
        sub_areas.append(Polygon(sub_area_coords))

# Create a GeoDataFrame with the sub-areas
sub_areas_gdf = geopandas.GeoDataFrame(geometry=sub_areas)

# Perform the spatial intersection to obtain the divided sub-areas
divided_areas = geopandas.overlay(sub_areas_gdf, rectangle, how='intersection')

# print(divided_areas)

for index, row in divided_areas.iterrows():
    geojson = row.geometry.__geo_interface__
    folium.GeoJson(geojson).add_to(m)
    
centroids = set()

for index, row in divided_areas.iterrows():    
    if(polygon_geom.intersects(row.geometry).bool()==True):
        centroids.add(row.geometry.centroid)

for centroid in centroids:
    # print(centroid)
    folium.GeoJson(centroid).add_to(m)

# folium.GeoJson(divided_areas.to_json).add_to(m)

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))

m

### The class interface is created and the method get_weather_forecast is called by group 6

In [3]:
interface = Interface()
interface.get_weather_forecast(polygon_geom)

[                 time  AOI_ID  EventID  temperature_2m  relativehumidity_2m  \
 0    2023-05-17T00:00       1        1             8.8                   74   
 1    2023-05-17T01:00       1        1             8.8                   74   
 2    2023-05-17T02:00       1        1             8.4                   73   
 3    2023-05-17T03:00       1        1             8.3                   70   
 4    2023-05-17T04:00       1        1             8.0                   68   
 ..                ...     ...      ...             ...                  ...   
 163  2023-05-23T19:00       1        1            11.3                   76   
 164  2023-05-23T20:00       1        1            11.2                   80   
 165  2023-05-23T21:00       1        1            11.1                   82   
 166  2023-05-23T22:00       1        1            10.9                   81   
 167  2023-05-23T23:00       1        1            10.8                   78   
 
      dewpoint_2m  apparent_temperatur